In [1]:
# pip install yfinance

In [2]:
# pip install plotly

In [3]:
import yfinance as yf
import plotly.graph_objects as go
import numpy as np
import pandas as pd
from datetime import date

In [4]:
# RMA function
def rma(input_data, input_window_length):
    alpha = 1/input_window_length
    rma_list = []
    for i in range(len(input_data)):
        if i < input_window_length:
            rma_list.append(None)
        elif i == input_window_length:
            rma_list.append(np.mean(input_data[i-input_window_length:i])) #SMA initial
        else:
            rma_list.append(alpha*input_data[i] + (1 - alpha)*rma_list[i-1])
    return rma_list

In [5]:
# True Range Function
def true_range(input_data):
    true_atr_list = []
    for i in range(len(input_data)):
        if i == 0:
            true_atr_list.append(input_data.High[i] - input_data.Low[i])
        else:
            high_low = input_data.High[i] - input_data.Low[i]
            high_close = input_data.High[i] - input_data.Close[i-1]
            low_close = input_data.Low[i] - input_data.Close[i-1]
            true_atr_list.append(max(max(high_low, abs(high_close)), abs(low_close)))
    return true_atr_list

In [6]:
# ATR function
def atr(input_data, input_window_length):
    return rma(true_range(input_data), input_window_length)

In [7]:
# v_stop function
def vstop(input_data, input_window_length, input_atr_factor):
    atr_list = []
    true_range_list = []
    atrM_list = []
    vstop_list = []
    uptrend_list = []
    max_min_list = []
    
    atr_list = atr(input_data, input_window_length)
    true_range_list = true_range(input_data)
    
    for i in range(len(input_data)):
        if i < input_window_length:
            atrM_list.append(true_range_list[i])
        else:
            atrM_list.append(atr_list[i]*input_atr_factor)
       
    for i in range(len(input_data)): 
        if i == 0: #base case up trend
            vstop_list.append(input_data.Close[i])
            uptrend_list.append(True)
            max_min_list.append(input_data.Close[i])
        elif i == 1:
            vstop_list.append(max(vstop_list[i-1], max_min_list[i-1] - atrM_list[i]))
            if input_data.Close[i] - vstop_list[i] >= 0:
                uptrend_list.append(True)
                max_min_list.append(max(max_min_list[i-1], input_data.Close[i]))
            else:
                vstop_list[i] = input_data.Close[i] + atrM_list[i]
                uptrend_list.append(False)
                max_min_list.append(input_data.Close[i])
        else: 
            if uptrend_list[i-1] == True:
                vstop_list.append(max(vstop_list[i-1], max_min_list[i-1] - atrM_list[i]))
                if input_data.Close[i] - vstop_list[i] >= 0:
                    uptrend_list.append(True)
                    max_min_list.append(max(max_min_list[i-1], input_data.Close[i]))
                else:
                    vstop_list[i] = input_data.Close[i] + atrM_list[i]
                    uptrend_list.append(False)
                    max_min_list.append(input_data.Close[i])
            else:
                vstop_list.append(min(vstop_list[i-1], max_min_list[i-1] + atrM_list[i]))
                if input_data.Close[i] - vstop_list[i] <= 0:
                    uptrend_list.append(False)
                    max_min_list.append(min(max_min_list[i-1], input_data.Close[i]))
                else:
                    vstop_list[i] = input_data.Close[i] - atrM_list[i]
                    uptrend_list.append(True)
                    max_min_list.append(input_data.Close[i])

    return [vstop_list, uptrend_list]

In [8]:
# Define Global Variable
atr_length = 10
atr_factor = 4.0
start_date = "1990-01-01" # Start Date
end_date = date.today() # End Date

In [9]:
# User Define
tickers = ["TSLA", "AAPL"] # Tickers List

In [10]:
vstop_result = []
vstop_result_list = []
v_change = []
for ticker in tickers:
    downloaded_data = yf.download(ticker, start = start_date, end = end_date)
    vstop_result = vstop(downloaded_data,atr_length,atr_factor)
    if vstop_result[1][len(vstop_result[1]) == 2:len(vstop_result[1])]:
        v_change.append(False)
    else:
        v_change.append(True)
    vstop_result_list.append(vstop_result[1][len(vstop_result[1])-2:len(vstop_result[1])])
target = pd.DataFrame(vstop_result_list, columns = ["Previous Day", "Today"], index = tickers)
target["V Change"] = pd.DataFrame({"V Change":v_change}, index = tickers)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [11]:
# Output
print(target[target["V Change"] == True].index)

Index([], dtype='object')
